<a href="https://colab.research.google.com/github/MaherABBASSI/Dog_rating_wrangling_Udacity/blob/master/Flower_resnet152.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget 'https://www.dropbox.com/s/9tffplt24nanxnf/flowers_test.zip?dl=0'

--2019-01-01 09:02:55--  https://www.dropbox.com/s/9tffplt24nanxnf/flowers_test.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9tffplt24nanxnf/flowers_test.zip [following]
--2019-01-01 09:02:56--  https://www.dropbox.com/s/raw/9tffplt24nanxnf/flowers_test.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc01eaf091fb14278fc2713198ce.dl.dropboxusercontent.com/cd/0/inline/AYlGHJSe1rBHoLUh4Qu5Wn0rTJsLhEUZoXb2IaDwtuAnwKJpb2aeXKta6_08wX-XrP1vuFNita4ZX6qPRk1bgS-5pDWIjWeLmy3W1XkzSr0wNLRByxQfOMS2d9__tAfKobrMIq7bEi2miRuFQAb9MxgOw1GDEYLdUpAxQND2AWYKrGJ7KPz3wVxTL3snKOHQ1Zg/file [following]
--2019-01-01 09:02:56--  https://uc01eaf091fb14278fc2713198ce.dl.dropboxusercontent.com/cd/0/inline/AYlGHJSe1rBHoLUh4Qu5Wn0rTJsLhEU

In [0]:
!unzip 'flowers_test.zip?dl=0' -d 'flowers_test'

Archive:  flowers_test.zip?dl=0
replace flowers_test/flowers_test/flower_data/test/1/image_06743.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
!wget 'https://www.dropbox.com/s/1axw0f86uw2gn9u/model_BN1.pt?dl=0'

--2019-01-01 09:04:26--  https://www.dropbox.com/s/1axw0f86uw2gn9u/model_BN1.pt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/1axw0f86uw2gn9u/model_BN1.pt [following]
--2019-01-01 09:04:27--  https://www.dropbox.com/s/raw/1axw0f86uw2gn9u/model_BN1.pt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccee0524c41a264503ca1e0d04a.dl.dropboxusercontent.com/cd/0/inline/AYm-Zgom8lRluu2lvyn9fOZ_jg34QSkG-92X1WdrgjaJShnBIorc8t44D3UFKcZqftJKRqkb2iEXqy7tWRk_yl44xqqohnny42i-3PDBa6m9o6LyWjPgcnurSYKEwKUvX8ttzxqggJ14n21JdyIpybz8ZzDMvGvxkViT1VqsTtvJqj8lzhn05rbJvs7xtnTP1Xg/file [following]
--2019-01-01 09:04:27--  https://uccee0524c41a264503ca1e0d04a.dl.dropboxusercontent.com/cd/0/inline/AYm-Zgom8lRluu2lvyn9fOZ_jg34QSkG-92X1Wdrgja

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x5ceee000 @  0x7f04829682a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import matplotlib.pyplot as plt

In [0]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
# TODO: Define your transforms for the training and validation sets
data_transforms = transforms.Compose([transforms.RandomResizedCrop(224), 
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(30), 
                                transforms.ToTensor(), 
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(r'flowers_test/flowers_test/flower_data/train', transform = data_transforms)
valid_data = datasets.ImageFolder(r'flowers_test/flowers_test/flower_data/valid', transform = data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
batch_size = 20
num_workers= 20
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)

In [0]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [0]:
# Load the pretrained model from pytorch
model = models.resnet152(pretrained=True)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # linear layer (2048 -> 1024)
        self.fc1 = nn.Linear(2048, 1024)
        self.batch1 = nn.BatchNorm1d(1024)
        # linear layer (1024 -> 512)
        self.fc2 = nn.Linear(1024, 512)
        self.batch2 = nn.BatchNorm1d(512)
        # linear layer (512 -> 102)
        self.fc3 = nn.Linear(512, 102)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.batch1(self.fc1(x)))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = F.relu(self.batch2(self.fc2(x)))
        # add dropout layer
        x = self.dropout(x)
        # add 3d hidden layer
        x = self.fc3(x)
        return x

# create a complete CNN
sub_model = Net()
print(sub_model)

Net(
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (batch1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (batch2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=102, bias=True)
  (dropout): Dropout(p=0.25)
)


In [0]:
model.fc = sub_model

In [0]:
for name, param in model.named_parameters():
  param.requires_grad = False

for name, param in model.fc.named_parameters():
  param.requires_grad = True


In [0]:
for name, param in model.named_parameters():
  param.requires_grad = True

In [0]:
import torch.optim as optim
from torch.optim import lr_scheduler

if train_on_gpu:
    model.cuda()
    
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(model.fc.parameters(),  lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [0]:
import torch.optim as optim
from torch.optim import lr_scheduler

if train_on_gpu:
    model.cuda()
    
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(model.parameters(),  lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.01)

In [0]:
pretrained_dict = torch.load('model_BN1.pt?dl=0', map_location=lambda storage, loc: storage)
model_dict = model.state_dict()

# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict) 
# 3. load the new state dict
model.load_state_dict(pretrained_dict)


In [0]:
!pip install Pillow==4.1.1
!pip install image

    100% |████████████████████████████████| 5.7MB 7.0MB/s 
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
# number of epochs to train the model
n_epochs = 200

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the mode
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    if (batch_idx+1) % 100 == 0:
        scheduler.step()
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.092728 	Validation Loss: 0.162830
Validation loss decreased (inf --> 0.162830).  Saving model ...
Epoch: 2 	Training Loss: 0.088015 	Validation Loss: 0.138633
Validation loss decreased (0.162830 --> 0.138633).  Saving model ...
Epoch: 3 	Training Loss: 0.115994 	Validation Loss: 0.147121
Epoch: 4 	Training Loss: 0.106368 	Validation Loss: 0.131873
Validation loss decreased (0.138633 --> 0.131873).  Saving model ...
Epoch: 5 	Training Loss: 0.106164 	Validation Loss: 0.140383
Epoch: 6 	Training Loss: 0.095541 	Validation Loss: 0.165790
Epoch: 7 	Training Loss: 0.091531 	Validation Loss: 0.154355
Epoch: 8 	Training Loss: 0.100580 	Validation Loss: 0.147570
Epoch: 9 	Training Loss: 0.095488 	Validation Loss: 0.131907
Epoch: 10 	Training Loss: 0.086136 	Validation Loss: 0.131151
Validation loss decreased (0.131873 --> 0.131151).  Saving model ...
Epoch: 11 	Training Loss: 0.087890 	Validation Loss: 0.129441
Validation loss decreased (0.131151 --> 0.129441).  Savi

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))